O Literal do módulo typing no Python é uma funcionalidade que permite especificar que uma variável ou um parâmetro de função só pode aceitar um valor exato (literal), e não apenas um tipo de dado.

Em outras palavras, ele transforma o valor em um tipo.

💡 O Conceito de Literal
Normalmente, você usa str para indicar que uma variável é uma string. Ao usar Literal["vermelho", "azul"], você está dizendo: "Esta variável não é apenas uma string; ela só pode ser a string 'vermelho' ou a string 'azul'."

In [ ]:
from typing import TypedDict, Literal

class TypedDictState(TypedDict):
    name: str
    mood: Literal["happy","sad"]

sempre que você usa o builder.add_conditional_edges(), a única maneira de definir para qual nó o fluxo deve seguir é através do valor de retorno (return) da função condicional que você passa como segundo argumento.

A função passada para add_conditional_edges (como o seu decide_mood ou a função should_continue em outros exemplos) tem uma responsabilidade simples e crucial: retornar o nome (string) do próximo nó.

In [ ]:
import random
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

def node_1(state):
    print("---Node 1---")
    return {"name": state['name'] + " is ... "}

def node_2(state):
    print("---Node 2---")
    return {"mood": "happy"}

def node_3(state):
    print("---Node 3---")
    return {"mood": "sad"}

def decide_mood(state) -> Literal["node_2", "node_3"]:
        
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"

# Build graph
builder = StateGraph(TypedDictState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()


## Dataclass

As dataclass oferecem uma sintaxe concisa para a criação de classes que são usadas principalmente para armazenar dados.

Exemplo:

In [ ]:
from dataclasses import dataclass

@dataclass
class DataclassState:
    name: str
    mood: Literal["happy","sad"]

### Acesso aos Atributos (.name vs ["name"])

A principal mudança ao usar dataclass em vez de um TypedDict é a forma como você acessa os valores dentro da função (nó) do LangGraph:

Com TypedDict: Você acessaria os valores como em um dicionário: state["name"].

Com Dataclass: Você acessa os valores como atributos de classe: state.name.

O exemplo da função node_1 demonstra isso:

~~~Python

def node_1(state):
    print("---Node 1---")
    return {"name": state.name + " is ... "}

~~~

O nó usa state.name para ler o valor atual de name do objeto state.


### Atualizações de Estado (O "Estranho")
Este é o ponto mais importante e, como o texto menciona, "um pouco estranho" (a nuance do LangGraph):

A "Estranheza": Mesmo que o estado (state) agora seja um objeto DataclassState (acessado via state.name), o nó node_1 ainda retorna um dicionário ({"name": ...}).

Por Quê? LangGraph armazena as chaves do seu estado (seja ele TypedDict ou Dataclass) separadamente. O sistema de atualização do LangGraph é projetado para fazer updates incrementais e imutáveis.

Regra de Update: O objeto retornado pelo nó (o dicionário) só precisa ter chaves (atributos) que correspondam aos definidos no estado (DataclassState tem a chave name).

Conclusão: Ao retornar {"name": ...}, você está dizendo ao LangGraph: "Atualize apenas o valor da chave name no estado atual com este novo valor." Os outros atributos (como mood) permanecem inalterados.

Portanto, o LangGraph aceita tanto a leitura orientada a objeto (state.name) quanto a atualização orientada a dicionário (return {"name": ...}) para manter a consistência do fluxo de atualização de estado.

In [ ]:
def node_1(state):
    print("---Node 1---")
    return {"name": state.name + " is ... "}

# Build graph
builder = StateGraph(DataclassState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

## Pydantic

### O Limite do TypedDict e Dataclasses
O ponto crucial aqui é a distinção entre Type Hinting e Validação em Runtime:

TypedDict e dataclasses fornecem apenas Type Hints (Dicas de Tipo).

Elas dizem aos verificadores de tipo estático (como MyPy) e às IDEs qual tipo é esperado (ex: str, int, Literal["happy", "sad"]).

No entanto, elas não impõem essas regras de tipo durante a execução (runtime).

O Risco: O código mostra que você pode ignorar a dica de tipo e atribuir um valor inválido, e o Python não reclamará.


### A Vantagem do Pydantic
Pydantic resolve essa falha, atuando como uma camada de validação de dados poderosa:

Validação em Runtime: Pydantic usa as anotações de tipo do Python, mas garante que os dados fornecidos realmente se conformem aos tipos e restrições especificados no momento em que o objeto é criado.

Melhor para LangGraph: Sua capacidade de validar e garantir a integridade dos dados o torna ideal para definir o esquema de estado (state schemas) no LangGraph. Se um nó tentar retornar um estado com um tipo ou valor inválido, o Pydantic levantará um erro imediatamente, impedindo que dados inconsistentes corrompam o grafo.

Comportamento: Se você tentasse criar uma instância de um modelo Pydantic com mood="mad", ele lançaria um erro de validação (ex: ValidationError), obrigando você a corrigir o dado antes que ele fosse processado.

Este bloco de código demonstra como a biblioteca Pydantic é usada para definir uma estrutura de dados (PydanticState) e, crucialmente, como ela impõe regras de validação (validadores personalizados) durante a criação da instância.

1. Definição do Modelo de Estado (PydanticState)

~~~Python

from pydantic import BaseModel, field_validator, ValidationError

class PydanticState(BaseModel):
    name: str
    mood: str # "happy" or "sad" 
~~~

* BaseModel: A classe PydanticState herda de BaseModel, o que a transforma em um modelo Pydantic. Isso significa que ela ganha automaticamente recursos como validação, serialização (para JSON) e documentação baseada nas type hints do Python.

* Campos: Os campos name e mood são definidos como str. Embora a type hint seja str, o comentário (# "happy" or "sad") indica uma restrição adicional que será imposta pelo validador.

2. Validação Personalizada (@field_validator)
~~~Python

    @field_validator('mood')
    @classmethod
    def validate_mood(cls, value):
        # Ensure the mood is either "happy" or "sad"
        if value not in ["happy", "sad"]:
            raise ValueError("Each mood must be either 'happy' or 'sad'")
        return value
~~~

@field_validator('mood'): Este decorador registra o método validate_mood para ser executado especificamente no campo mood sempre que uma nova instância de PydanticState for criada.

Lógica de Validação:

* O validador recebe o valor (value) que está sendo atribuído a mood.

* Ele verifica se o value está contido na lista permitida ["happy", "sad"].

* Se o valor for inválido ("mad", por exemplo), ele levanta um ValueError com uma mensagem explicativa.

* Se o valor for válido, ele deve retornar o valor (return value) para que o processo de criação da instância continue.

3. Teste de Validação em Runtime
~~~Python

try:
    state = PydanticState(name="John Doe", mood="mad")
except ValidationError as e:
    print("Validation Error:", e)

~~~

* Tentativa de Criação: O código tenta criar uma instância de PydanticState, passando o valor inválido "mad" para o campo mood.

* Ação do Pydantic: Ao receber o valor "mad", o Pydantic executa o validate_mood.

* Lançamento do Erro: O validate_mood detecta que "mad" não está na lista permitida e levanta o ValueError.

* Captura do Erro: O Pydantic captura esse ValueError e o encapsula em seu próprio tipo de erro, o ValidationError.

* Resultado: O bloco except é executado, e a mensagem de erro é impressa no terminal. Isso confirma que o Pydantic impediu a criação de um objeto de estado com dados inconsistentes, demonstrando sua função principal de validação em tempo de execução.

In [ ]:
from pydantic import BaseModel, field_validator, ValidationError

class PydanticState(BaseModel):
    name: str
    mood: str # "happy" or "sad" 

    @field_validator('mood')
    @classmethod
    def validate_mood(cls, value):
        # Ensure the mood is either "happy" or "sad"
        if value not in ["happy", "sad"]:
            raise ValueError("Each mood must be either 'happy' or 'sad'")
        return value

try:
    state = PydanticState(name="John Doe", mood="mad")
except ValidationError as e:
    print("Validation Error:", e)

In [ ]:
# Build graph
builder = StateGraph(PydanticState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(PydanticState(name="Lance",mood="sad"))